In [1]:
import sys
import numpy as np
import pickle as pkl
from IPython.display import display
from maraboupy import Marabou, MarabouCore
from utils import TOTUtils

In [2]:
nnet_path = '../artifacts/models/model_v3.2.0/model.h5'
test_data_csv = '../data/v3.2.0/test.csv'
all_samples = TOTUtils.load_samples(test_data_csv, frac=0.01)
samples = TOTUtils.filter_samples(all_samples, nnet_path)
print(f'{len(samples)} correct samples')

1329 correct samples


In [3]:
# # Make sure that all of the sample inputs return correct category.
# from tot_net import TOTNet

# x=0
# dists=(0,0)
# res = []
# for i,sample in enumerate(samples):
#     inputs, outputs = sample
#     exp_cat = outputs.index(max(outputs))
#     other_cats = [i for i in range(5) if i != exp_cat]
#     net = TOTNet(nnet_path)
#     net.set_lower_bounds(inputs)
#     net.set_upper_bounds(inputs)
#     net.set_input_lower_bound(x, inputs[x]+dists[0])
#     net.set_input_upper_bound(x, inputs[x]+dists[1])
#     res.append([])
#     for oc in other_cats:
#         net.set_expected_category(oc)
#         r,_ = net.solve()
#         res[-1].append(r)

# wrong_preds = [ir for r in res for cr in r for ir in cr if len(ir) > 0]
# assert(len(wrong_preds) == 0)

In [4]:
# from sensitivity import find_feature_sensitivity_boundaries

# find_feature_sensitivity_boundaries('../artifacts/models/model_v3.2.0/model.h5', 0, samples[::], d_min=0.00001, d_max=50, multithread=True, verbose=1)

In [5]:
from sensitivity import find_sensitivity_boundaries, save_sensitivity_results_to_csv

h5_path = '../artifacts/models/model_v3.2.0/model.h5'
results = find_sensitivity_boundaries(h5_path, samples, d_min=0.00001, d_max=100, multithread=True, save_results=True)

for x,result in results.items():
    dists,_ = result
    print(x, dists)

In [6]:
# from tensorflow.keras.models import load_model

# model = load_model('../artifacts/models/model_v3.2.0/model.h5')
# inputs,outputs = samples[6]
# d = 49.69331
# x = 0
# pred = model.predict([inputs[0:x] + [inputs[x]+d] + inputs[x+1:]])

# # assert that prediction was correct
# assert(outputs.index(max(outputs)) == list(pred[0]).index(max(pred[0])))

In [7]:
# from maraboupy import Marabou, MarabouCore

# pb_path = '../artifacts/models/model-v3.1.4/model.pb'
# net = Marabou.read_tf(pb_path)

# sample = samples[0]
# inputs, outputs = sample
# expected_cat = outputs.index(max(outputs))
# other_cats = [i for i in range(len(outputs)) if i != expected_cat]

# for x,v in enumerate(inputs):
#     net.setLowerBound(net.inputVars[0][x], v)
#     net.setUpperBound(net.inputVars[0][x], v)
# for c in other_cats:
#     print('OC:', c)
#     eq1 = MarabouCore.Equation(MarabouCore.Equation.LE)
#     eq1.addAddend(1, net.outputVars[0][c])
#     eq1.addAddend(-1, net.outputVars[0][0])
#     eq1.setScalar(0)
# pred = net.evaluate([input_sample], useMarabou=False)
# print(pred)